In [ ]:
import pandas as pd

def klines(symbol, interval, start):
    params = {
        "symbol" : symbol,
        "interval" : interval,
        "startTime" : start
    }
    url = urljoin(base_url, endppoint)
    r = requests.get(url, params=params)
    code = r.status_code
    print("code : ",code)
    if code == 200:
        return r.json()
    

X = klines("BTCUSDT", interval="1d", start="2023")

col = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'num_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
df = pd.DataFrame(X, columns = col)
df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
df

In [ ]:
from datetime import datetime
import time

import requests
import hmac
import hashlib
from urllib.parse import urljoin, urlencode

import concurrent.futures

from dataEngine.extract import Binance


%load_ext autoreload
%autoreload 2

"GET /fapi/v1/continuousKlines"

base_url = "https://testnet.binancefuture.com"
endppoint = "/fapi/v1/klines"


def klines(symbol, interval, start):
    params = {
        "symbol" : symbol,
        "interval" : interval,
        "startTime" : start
    }
    url = urljoin(base_url, endppoint)
    r = requests.get(url, params=params)
    code = r.status_code
    print("code : ",code)
    if code == 200:
        return r.json()



all_data = {}

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(klines, symbol, interval, start) for symbol in symbols]
    
    for future in concurrent.futures.as_completed(futures):
        symbol, data = future.result()
        all_data[symbol] = data

if all_data:
    for symbol, data in all_data.items():
        print(f" symbol {symbol} : {data}")
 



In [ ]:
from TCed.agent import Agent
from TCed.globaleAgent import GlobalAgent
from TCed.env import Env

import multiprocessing

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

config = {"capital" : 100, "size" : 45, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2022", "end" : "2023"
          }


env = Env(**config)

master = multiprocessing.Queue()
agents = multiprocessing.Queue()

ENV = multiprocessing.Queue()
ENV.put(env)

communication_lock = multiprocessing.Condition()

In [ ]:
from dataEngine.data import connect_db

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


SYMBOLS = ["DASH", "ETH", "XMR"]
CAPITAL = 100
START = "2023"
END = "2023"

datas = []

db = connect_db(name = "database", interval = "1d")

def gen_data(symbol, size = 10):
    i = 0
    data = db.get_data(symbol, start = START, end = END)
    while  True:
        batch = data.iloc[i : i + size]
        if len(batch) == size:
            yield batch
        else:
            break
        i += 1

x = gen_data("BTC")
x

In [ ]:
next(x)

# GenData

In [ ]:
import zmq
from IPython.display import clear_output
from dataEngine.data import connect_db

In [ ]:
SYMBOLS = ["DASH", "TWT"]
START = "2023-01"
END = "2023-01"

db = connect_db(name = "database", interval = "1d")
data = db.get_data(SYMBOLS[0], start = START, end = END)

In [ ]:
class Server:
    
    def __init__(self, symbols, start, end):
        self.symbols = symbols
        self.start = start 
        self.end = end
        
        self.get_data()
        self.create()
        
    
    def get_data(self):
        self.data = {}
        for symbol in self.symbols:
            data = db.get_data(symbol, start = self.start, end = self.end)
            self.data[symbol] = data
        
        
    def create(self):
        context = zmq.Context()
        self.socket = context.socket(zmq.PUB)
        self.socket.bind('tcp://127.0.0.1:8080')
        
    
    def line(self, symbol, data, bar):
        try:
            date = data.index[bar].timestamp()
            #date = date.timestamp()
            msg = '{} {} {} {} {} {} {}'.format(symbol, date,
                                            data.iloc[bar]['open'], data.iloc[bar]['high'],
                                            data.iloc[bar]['low'], data.iloc[bar]['close'],
                                            data.iloc[bar]['volume'])
        except:
            msg = "-"
            
        print(msg)
        self.socket.send_string(msg)
    
    
    def run(self):
        bar = 0
        while True:
            clear_output(True)
            for symbol in self.symbols:
                self.line(symbol, self.data.get(symbol), bar)
        
        bar += 1
        print("bar : ",bar)
        

In [ ]:
SYMBOLS = ["BTC", "ETH"]
START = "2023-01"
END = "2023-01"


server = Server(symbols = SYMBOLS, start=START, end=END)
server.run()

# GlobalAgent

In [ ]:
import random
import multiprocessing


class GlobalAgent(multiprocessing.Process):
    
    def __init__(self, activeAgent = "", communication_lock="", agents="", master=""):
        multiprocessing.Process.__init__(self)
        self.activeAgent = activeAgent
        self.n_agent = 2
        
        self.agents = agents
        self.master = master
        
        self.communication_lock = communication_lock
                        
            
    def run(self):
        while True:
            
            with self.communication_lock:
                print(f" agents size {self.agents.qsize()}")
                self.communication_lock.wait_for(lambda : self.agents.qsize() == self.n_agent)
                print(f" agents size {self.agents.qsize()}")
                print("go the ----> ")
                
                for i in range(self.n_agent):
                    msg = self.agents.get()
                
                agent_id = random.randint(1, 4)
                self.activeAgent.value = agent_id
                self.master.put(msg)
                
                self.communication_lock.notify_all()
                
